In [1]:
import os

import time

import requests

from requests.adapters import HTTPAdapter

from urllib3.util.retry import Retry
 
# List of NSE tickers (add all you want here)

tickers = [

   "EGAD","KAPC","KUKZ","LIMT","SASN","WTK",
"CGEN",
"ABSA","SBIC","IMH","DTK","SCBK","EQTY","COOP","BKG","HFCK","KCB","NCBA",
"XPRS","SMER","KQ","NMG","SGL","TPSE","SCAN","UCHM","LKL","NBV",
"BAMB","CRWN","CABL","PORT",
"TOTL","KEGN","KPLC","UMME",
"JUB","SLAM","KNRE","LBTY","BRIT","CIC",
"OCH","CTUM","TCL","HAFR","KURV",
"NSE",
"BOC","BAT","CARB","EABL","UNGA","EVRD","AMAC","FTGH","SKL",
"SCOM",
"LAPR","GLD","SMWF"
]
 
BASE_URL = "https://afx.kwayisi.org/nse/{}.html"
 
os.makedirs("html", exist_ok=True)
 
# Session with retries

session = requests.Session()

retries = Retry(

    total=5,

    backoff_factor=2,

    status_forcelist=[429, 500, 502, 503, 504],

    allowed_methods=["GET"]

)

adapter = HTTPAdapter(max_retries=retries)

session.mount("https://", adapter)

session.mount("http://", adapter)
 
HEADERS = {"User-Agent": "Mozilla/5.0"}
 
for i, ticker in enumerate(tickers, 1):

    url = BASE_URL.format(ticker.lower())

    print(f"[{i}/{len(tickers)}] Downloading {ticker}...")
 
    try:

        r = session.get(url, headers=HEADERS, timeout=30)

        r.raise_for_status()
 
        filename = f"html/{ticker}.html"

        with open(filename, "w", encoding="utf-8") as f:

            f.write(r.text)
 
        print(f"✅ Saved {filename}")
 
    except Exception as e:

        print(f"⚠️ Failed to download {ticker}: {e}")
 
    time.sleep(3)  # be polite to the server
 
print("✅ Download step complete.")

[1/61] Downloading EGAD...


/Users/johngitau/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


✅ Saved html/EGAD.html
[2/61] Downloading KAPC...
✅ Saved html/KAPC.html
[3/61] Downloading KUKZ...
✅ Saved html/KUKZ.html
[4/61] Downloading LIMT...
✅ Saved html/LIMT.html
[5/61] Downloading SASN...
✅ Saved html/SASN.html
[6/61] Downloading WTK...
✅ Saved html/WTK.html
[7/61] Downloading CGEN...
✅ Saved html/CGEN.html
[8/61] Downloading ABSA...
✅ Saved html/ABSA.html
[9/61] Downloading SBIC...
✅ Saved html/SBIC.html
[10/61] Downloading IMH...
✅ Saved html/IMH.html
[11/61] Downloading DTK...
✅ Saved html/DTK.html
[12/61] Downloading SCBK...
✅ Saved html/SCBK.html
[13/61] Downloading EQTY...
✅ Saved html/EQTY.html
[14/61] Downloading COOP...
✅ Saved html/COOP.html
[15/61] Downloading BKG...
✅ Saved html/BKG.html
[16/61] Downloading HFCK...
✅ Saved html/HFCK.html
[17/61] Downloading KCB...
✅ Saved html/KCB.html
[18/61] Downloading NCBA...
✅ Saved html/NCBA.html
[19/61] Downloading XPRS...
✅ Saved html/XPRS.html
[20/61] Downloading SMER...
✅ Saved html/SMER.html
[21/61] Downloading KQ...


In [ ]:
import os
import re
import pandas as pd
from bs4 import BeautifulSoup

results = []
history_rows = []
html_folder = "html"


def parse_number(s):
    """Convert strings like '1.32T', '158B', '1.89M', '623,465' to float."""
    if not s or s == "—":
        return None
    s = s.strip().replace(",", "")
    multipliers = {"T": 1e12, "B": 1e9, "M": 1e6, "K": 1e3}
    for suffix, mult in multipliers.items():
        if s.upper().endswith(suffix):
            try:
                return float(s[:-1]) * mult
            except ValueError:
                return None
    try:
        return float(s)
    except ValueError:
        return None


def parse_pct(s):
    """Convert '+12.8%' or '-1.45%' to float like 12.8 or -1.45."""
    if not s:
        return None
    m = re.search(r"([+-]?[\d.]+)%", s)
    return float(m.group(1)) if m else None


for filename in sorted(os.listdir(html_folder)):
    if not filename.endswith(".html"):
        continue

    ticker = filename.replace(".html", "")
    filepath = os.path.join(html_folder, filename)

    with open(filepath, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f.read(), "html.parser")

    text = soup.get_text(" ", strip=True)

    # --- Current price & daily change ---
    price_match = re.search(
        rf"{ticker}\s*[•·]\s*([\d.]+)\s*[▴▾▵▿]?\s*([+-]?[\d.]+)\s*\(([\d.]+%)\)",
        text, re.IGNORECASE,
    )
    current_price = float(price_match.group(1)) if price_match else None
    daily_change = float(price_match.group(2)) if price_match else None
    daily_change_pct = parse_pct(price_match.group(3)) if price_match else None

    # --- Company name from <h1> ---
    h1 = soup.find("h1")
    company_name = h1.text.split(" - ", 1)[1].strip() if h1 and " - " in h1.text else None

    # --- Live Trading Feed table ---
    def get_table_value(label):
        """Find a <td> whose text matches label, return the next <td>'s text."""
        for td in soup.find_all("td"):
            if td.get_text(strip=True) == label:
                nxt = td.find_next_sibling("td")
                if nxt:
                    return nxt.get_text(strip=True)
        return None

    opening = parse_number(get_table_value("Opening Price"))
    day_low = parse_number(get_table_value("Day's Low Price"))
    day_high = parse_number(get_table_value("Day's High Price"))
    volume = parse_number(get_table_value("Traded Volume"))
    num_deals = parse_number(get_table_value("Number of Deals"))
    turnover = parse_number(get_table_value("Gross Turnover"))

    # --- Growth & Valuation table ---
    eps = parse_number(get_table_value("Earnings Per Share"))
    pe_ratio = parse_number(get_table_value("Price/Earning Ratio"))
    dps = parse_number(get_table_value("Dividend Per Share"))
    div_yield_raw = get_table_value("Dividend Yield")
    div_yield = parse_pct(div_yield_raw) if div_yield_raw else None
    shares_out = parse_number(get_table_value("Shares Outstanding"))
    market_cap = parse_number(get_table_value("Market Capitalization"))

    # --- Performance percentages (1WK, 4WK, 3MO, 6MO, 1YR, YTD) ---
    perf_div = soup.find("div", attrs={"data-perf": True})
    perf = {}
    if perf_div:
        headers = [th.get("title", th.text) for th in perf_div.find_all("th")]
        values = [td.get_text(strip=True) for td in perf_div.find_all("td")]
        for h, v in zip(headers, values):
            perf[h] = parse_pct(v)

    # --- Sector & Industry from factsheet ---
    sector = industry = None
    fact_div = soup.find("div", attrs={"data-fact": True})
    if fact_div:
        for dt in fact_div.find_all("dt"):
            label = dt.get_text(strip=True)
            dd = dt.find_next_sibling("dd")
            if dd:
                if label == "Sector":
                    sector = dd.get_text(strip=True)
                elif label == "Industry":
                    industry = dd.get_text(strip=True)

    # --- Historical 10-day prices ---
    hist_table = soup.find("table", attrs={"data-hist": True})
    if hist_table:
        for tr in hist_table.find("tbody").find_all("tr"):
            tds = [td.get_text(strip=True) for td in tr.find_all("td")]
            if len(tds) >= 3:
                history_rows.append({
                    "Ticker": ticker,
                    "Date": tds[0],
                    "Volume": parse_number(tds[1]),
                    "Close": parse_number(tds[2]),
                    "Change": parse_number(tds[3]) if len(tds) > 3 else None,
                    "Change%": parse_pct(tds[4]) if len(tds) > 4 else None,
                })

    results.append({
        "Ticker": ticker,
        "Company": company_name,
        "Sector": sector,
        "Industry": industry,
        "Price": current_price,
        "Change": daily_change,
        "Change%": daily_change_pct,
        "Open": opening,
        "Low": day_low,
        "High": day_high,
        "Volume": volume,
        "Deals": num_deals,
        "Turnover": turnover,
        "EPS": eps,
        "P/E": pe_ratio,
        "DPS": dps,
        "Div Yield%": div_yield,
        "Shares Out": shares_out,
        "Market Cap": market_cap,
        "1WK%": perf.get("1-Week"),
        "4WK%": perf.get("4-Week"),
        "3MO%": perf.get("3-Month"),
        "6MO%": perf.get("6-Month"),
        "1YR%": perf.get("1-Year"),
        "YTD%": perf.get("Year-to-Date"),
    })

    print(f"Parsed {ticker}: KES {current_price} | YTD {perf.get('Year-to-Date', 'N/A')}%")

df = pd.DataFrame(results)
df_hist = pd.DataFrame(history_rows)
df_hist["Date"] = pd.to_datetime(df_hist["Date"])

df.to_csv("nse_snapshot.csv", index=False)
df_hist.to_csv("nse_history_10d.csv", index=False)

print(f"\nExtracted {len(df)} stocks, {len(df_hist)} historical rows")
print("Saved: nse_snapshot.csv, nse_history_10d.csv")
df

## Accumulate Historical Data

Pivots the 10-day history from each HTML scrape into date-stamped columns (`Close_2026-02-11`, `Volume_2026-02-11`, ...).  
Merges with existing `nse_historical.csv` so data accumulates across runs — old dates are preserved, new dates are appended as columns.  
Run this notebook daily to build up your own price history over time.

In [ ]:
HIST_FILE = "nse_historical.csv"

# --- Build wide-format columns from the scraped 10-day history ---
# df_hist has: Ticker, Date, Volume, Close, Change, Change%
new_wide = df_hist.copy()
new_wide["DateStr"] = new_wide["Date"].dt.strftime("%Y-%m-%d")

# Pivot Close prices: one column per date
close_pivot = new_wide.pivot(index="Ticker", columns="DateStr", values="Close")
close_pivot.columns = [f"Close_{d}" for d in close_pivot.columns]

# Pivot Volumes: one column per date
vol_pivot = new_wide.pivot(index="Ticker", columns="DateStr", values="Volume")
vol_pivot.columns = [f"Volume_{d}" for d in vol_pivot.columns]

# Combine into one wide DataFrame
new_data = close_pivot.join(vol_pivot)

# Add static info columns from main df
static_cols = df.set_index("Ticker")[["Company", "Sector", "Industry"]]
new_data = static_cols.join(new_data)
new_data.index.name = "Ticker"

# --- Merge with existing historical file ---
if os.path.exists(HIST_FILE):
    existing = pd.read_csv(HIST_FILE, index_col="Ticker")
    print(f"Loaded existing {HIST_FILE}: {existing.shape[0]} tickers, {len(existing.columns)} columns")

    # Find date columns already in existing file
    existing_date_cols = [c for c in existing.columns if c.startswith("Close_") or c.startswith("Volume_")]
    new_date_cols = [c for c in new_data.columns if c.startswith("Close_") or c.startswith("Volume_")]

    # New dates that don't exist yet
    truly_new = [c for c in new_date_cols if c not in existing.columns]

    # Update: overwrite new data columns into existing, keep old ones intact
    for col in new_date_cols:
        existing[col] = new_data[col]

    # Also update static columns in case a new ticker was added
    for col in ["Company", "Sector", "Industry"]:
        existing[col] = new_data[col].combine_first(existing[col]) if col in existing.columns else new_data[col]

    # Add any new tickers that weren't in the file before
    new_tickers = new_data.index.difference(existing.index)
    if len(new_tickers) > 0:
        existing = pd.concat([existing, new_data.loc[new_tickers]])

    merged = existing
    print(f"Added {len(truly_new)} new date columns: {truly_new[:5]}{'...' if len(truly_new) > 5 else ''}")
else:
    merged = new_data
    print(f"No existing {HIST_FILE} found — creating new file")

# --- Sort columns: static first, then date columns in chronological order ---
static = ["Company", "Sector", "Industry"]
date_cols = sorted([c for c in merged.columns if c not in static])
merged = merged[static + date_cols]

# Save
merged.to_csv(HIST_FILE)

# Summary
close_dates = sorted(set(c.replace("Close_", "") for c in merged.columns if c.startswith("Close_")))
print(f"\nSaved {HIST_FILE}: {merged.shape[0]} tickers x {len(close_dates)} unique dates")
print(f"Date range: {close_dates[0]} to {close_dates[-1]}")
print(f"Total columns: {len(merged.columns)}")
print(f"\nRun this notebook again tomorrow to add more dates!")
merged.head()

## Momentum & Value Analysis

Ranks stocks by short-term momentum (1WK, 4WK) and longer-term trend (3MO, YTD).  
Calculates intraday volatility range and flags potential buy/sell signals.

In [ ]:
import numpy as np

analysis = df[["Ticker", "Company", "Sector", "Price", "Open", "Low", "High",
               "Volume", "Turnover", "Market Cap",
               "1WK%", "4WK%", "3MO%", "6MO%", "1YR%", "YTD%"]].copy()

# --- Intraday range as % of price (volatility proxy) ---
analysis["Intraday Range%"] = np.where(
    analysis["Price"] > 0,
    ((analysis["High"] - analysis["Low"]) / analysis["Price"] * 100).round(2),
    None,
)

# --- Momentum score: weighted average of performance periods ---
# Higher weight on recent periods for trading signals
analysis["Momentum Score"] = (
    analysis["1WK%"].fillna(0) * 0.30 +
    analysis["4WK%"].fillna(0) * 0.25 +
    analysis["3MO%"].fillna(0) * 0.20 +
    analysis["YTD%"].fillna(0) * 0.15 +
    analysis["6MO%"].fillna(0) * 0.10
).round(2)

# --- Rank by momentum ---
analysis["Momentum Rank"] = analysis["Momentum Score"].rank(ascending=False).astype(int)

# --- Simple signal based on momentum alignment ---
def get_signal(row):
    wk1 = row["1WK%"] or 0
    wk4 = row["4WK%"] or 0
    mo3 = row["3MO%"] or 0
    ytd = row["YTD%"] or 0

    # Strong buy: all timeframes positive and accelerating (1WK > 4WK avg)
    if wk1 > 0 and wk4 > 0 and mo3 > 0 and ytd > 0 and wk1 > (wk4 / 4):
        return "STRONG BUY"
    # Buy: mostly positive trend
    elif wk1 > 0 and wk4 > 0 and mo3 > 0:
        return "BUY"
    # Hold: mixed signals
    elif wk1 > 0 or wk4 > 0:
        return "HOLD"
    # Sell: negative short-term momentum
    elif wk1 < 0 and wk4 < 0:
        return "SELL"
    else:
        return "HOLD"

analysis["Signal"] = analysis.apply(get_signal, axis=1)

# Sort by momentum rank
analysis = analysis.sort_values("Momentum Rank")

# Display key columns
display_cols = ["Momentum Rank", "Ticker", "Company", "Price", "1WK%", "4WK%",
                "3MO%", "YTD%", "Momentum Score", "Intraday Range%", "Signal"]
print("=" * 80)
print("NSE MOMENTUM DASHBOARD - Sorted by Momentum Score")
print("=" * 80)
analysis[display_cols]

## 10-Day Price Trends & Volume Analysis

Shows which stocks are trending up/down over the last 10 trading days and highlights unusual volume spikes (potential breakout signals).

In [ ]:
import matplotlib.pyplot as plt

# --- 10-day price change for each stock ---
trend_summary = []
for ticker, grp in df_hist.groupby("Ticker"):
    grp = grp.sort_values("Date")
    if len(grp) >= 2:
        oldest_close = grp.iloc[0]["Close"]
        latest_close = grp.iloc[-1]["Close"]
        pct_10d = ((latest_close - oldest_close) / oldest_close * 100) if oldest_close else 0
        avg_vol = grp["Volume"].mean()
        max_vol = grp["Volume"].max()
        vol_spike = max_vol / avg_vol if avg_vol else 0  # >2 = unusual spike
        positive_days = (grp["Change"].fillna(0) > 0).sum()
        negative_days = (grp["Change"].fillna(0) < 0).sum()
        trend_summary.append({
            "Ticker": ticker,
            "10D Change%": round(pct_10d, 2),
            "Avg Volume": int(avg_vol),
            "Max Volume": int(max_vol),
            "Vol Spike Ratio": round(vol_spike, 1),
            "Up Days": int(positive_days),
            "Down Days": int(negative_days),
            "Win Rate%": round(positive_days / len(grp) * 100, 1),
        })

df_trend = pd.DataFrame(trend_summary).sort_values("10D Change%", ascending=False)
print("=" * 70)
print("10-DAY TREND SUMMARY")
print("=" * 70)
print("Vol Spike Ratio > 2.0 = unusual volume (possible breakout)")
print()
display(df_trend)

# --- Plot: 10-day price trend for top 6 movers ---
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()
top_movers = df_trend.head(6)["Ticker"].tolist()

for i, ticker in enumerate(top_movers):
    grp = df_hist[df_hist["Ticker"] == ticker].sort_values("Date")
    ax = axes[i]
    ax.plot(grp["Date"], grp["Close"], marker="o", linewidth=2)
    ax.fill_between(grp["Date"], grp["Close"], alpha=0.2)
    ax.set_title(f"{ticker} (10D: {df_trend[df_trend['Ticker']==ticker]['10D Change%'].values[0]:+.1f}%)")
    ax.tick_params(axis="x", rotation=45)
    ax.grid(True, alpha=0.3)

plt.suptitle("Top 6 Movers - 10 Day Price Trend", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.savefig("top_movers_10d.png", dpi=150, bbox_inches="tight")
plt.show()
print("Saved: top_movers_10d.png")

## Sector Breakdown & Market Overview

Market cap distribution by sector, and a final summary of top picks.

In [ ]:
# --- Sector breakdown ---
sector_summary = df.groupby("Sector").agg(
    Stocks=("Ticker", "count"),
    Total_MCap=("Market Cap", "sum"),
    Avg_YTD=("YTD%", "mean"),
    Avg_1WK=("1WK%", "mean"),
).round(2)
sector_summary["Total_MCap_B"] = (sector_summary["Total_MCap"] / 1e9).round(1)
sector_summary = sector_summary.sort_values("Total_MCap", ascending=False)

print("=" * 60)
print("SECTOR BREAKDOWN")
print("=" * 60)
display(sector_summary[["Stocks", "Total_MCap_B", "Avg_YTD", "Avg_1WK"]])

# --- Pie chart of market cap by sector ---
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

sector_mcap = df.groupby("Sector")["Market Cap"].sum().sort_values(ascending=False)
ax1.pie(sector_mcap, labels=sector_mcap.index, autopct="%1.0f%%", startangle=90)
ax1.set_title("Market Cap by Sector")

# --- Bar chart: YTD% by stock ---
ytd = df[["Ticker", "YTD%"]].dropna().sort_values("YTD%", ascending=True)
colors = ["green" if v > 0 else "red" for v in ytd["YTD%"]]
ax2.barh(ytd["Ticker"], ytd["YTD%"], color=colors)
ax2.set_xlabel("YTD %")
ax2.set_title("Year-to-Date Performance")
ax2.axvline(x=0, color="black", linewidth=0.5)
ax2.grid(True, alpha=0.3, axis="x")

plt.tight_layout()
plt.savefig("nse_overview.png", dpi=150, bbox_inches="tight")
plt.show()

# --- Final top picks ---
print("\n" + "=" * 60)
print("TOP PICKS SUMMARY")
print("=" * 60)

strong_buys = analysis[analysis["Signal"] == "STRONG BUY"][["Ticker", "Company", "Price", "Momentum Score", "1WK%", "YTD%"]]
buys = analysis[analysis["Signal"] == "BUY"][["Ticker", "Company", "Price", "Momentum Score", "1WK%", "YTD%"]]

if len(strong_buys) > 0:
    print("\nSTRONG BUY signals:")
    display(strong_buys)
if len(buys) > 0:
    print("\nBUY signals:")
    display(buys)

sells = analysis[analysis["Signal"] == "SELL"][["Ticker", "Company", "Price", "Momentum Score", "1WK%", "YTD%"]]
if len(sells) > 0:
    print("\nSELL signals (negative momentum across timeframes):")
    display(sells)

# --- Volume leaders (most liquid = easier to trade) ---
print("\nMost Liquid Stocks (by turnover - easiest to buy/sell):")
liquidity = df.nlargest(5, "Turnover")[["Ticker", "Company", "Price", "Turnover", "Volume", "Deals"]]
display(liquidity)

print("\nNote: These signals are based on momentum trends only.")
print("Always consider fundamentals (EPS, P/E, dividends) before trading.")